# COGS 108 - Data Checkpoint

# Names

- Jared (Ruotian) Chen
- Jimin Cheon
- Kane Gu
- Laurence D'Ercole
- Nisha Davankar

<a id='research_question'></a>
# Research Question

## TODO!!!! Social media
What attributes of a YouTube video affect its popularity/shareability(likes and/or shares)?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [ ]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Project Proposal (updated)

| Meeting Date | Meeting Time    | Completed Before Meeting                | Discuss at Meeting                            |
|--------------|-----------------|-----------------------------------------|-----------------------------------------------|
| 2/2          | 6:00 PM         | Project Proposal done                   | Roles, data wrangling                         |
| 2/9          | 6:00 PM         | Review feedback, Part of data wrangling | EDA ideas                                     |
| 2/16         | 6:00 PM         | Data wrangling and Partial EDA          | Discuss analysis and EDA                      |
| 3/2          | 6:00 PM         | Outline of Analysis and EDA             | Continue Analysis                             |
| 3/9          | 6:00 PM         | Finish Analysis                         | Ethics, Meeting with TA                       |
| 3/16         | 6:00 PM         | Draft done                              | Final review                                  |
| 3/19         | Before 11:59 PM | N/A                                     | Turn in Final Project & Group Project Surveys |